In [ ]:
%pip install pyscf

In [1]:
import datetime as dt

import numpy as np

import matplotlib as plt

from qiskit import QuantumCircuit, generate_preset_pass_manager
from qiskit.circuit.library import TwoLocal
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator, Batch, IBMRuntimeError
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.units import DistanceUnit

from scipy.optimize import minimize

import pyscf

In [2]:
use_simulator = False

In [3]:
service = QiskitRuntimeService(channel='ibm_quantum', token='624dc5d6d4363f602d6f41953f98aa482181ddf6db1929df6406f4befb19fa08ba67bd1d376680bb218be695bdd89d5bc7a7e5dd167bb93115a71b23ad8e75df')
hardware = service.backend(name='ibm_rensselaer')
simulator = AerSimulator.from_backend(backend=hardware)
pm = generate_preset_pass_manager(backend=hardware, optimization_level=3)

if use_simulator:
  backend = simulator
else:
  backend = hardware

In [4]:
def get_timestamp():
  return dt.datetime.now(dt.UTC).isoformat()

def transpile_and_apply(ansatz, hamiltonian, backend, optimization_level=3, passmanager=None):
  if not passmanager:
    passmanager = generate_preset_pass_manager(target=backend.target,
                                               optimization_level=optimization_level)
  isa_ansatz = passmanager.run(ansatz)
  isa_hamiltonian = hamiltonian.apply_layout(isa_ansatz.layout)

  return isa_ansatz, isa_hamiltonian

In [5]:
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

#### The geometric structure of water
![water.png](water.png)

In [6]:
def create_h2o_molecule_driver(bond_length=95.84, angle=np.radians(104.45)):
  oxygen = f"O 0 0 0"
  hydrogen1 = f"H {-1 * np.sin(angle) * bond_length} {np.cos(angle) * bond_length} 0"
  hydrogen2 = f"H {np.sin(angle) * bond_length} {np.cos(angle) * bond_length} 0"

  water = "; ".join([oxygen, hydrogen1, hydrogen2])

  driver = PySCFDriver(
    atom=water,
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM
  )

  return driver

In [7]:
water_driver = create_h2o_molecule_driver()
problem = water_driver.run()

In [ ]:
hamiltonian = problem.hamiltonian.second_q_op()
print(hamiltonian)

In [9]:
def map_to_qubits(hamiltonian):
  mapper = JordanWignerMapper()
  mapped_hamiltonian = mapper.map(hamiltonian)

  return mapped_hamiltonian, mapper

In [ ]:
def create_ansatz(mapped_hamiltonian, mapper):
  num_spatial_orbitals = 7
  num_ab_particles = (5, 5)

  uccsd_ansatz = UCCSD(num_spatial_orbitals=num_spatial_orbitals,
                       num_particles=num_ab_particles,
                       qubit_mapper=mapper,
                       initial_state=HartreeFock(num_spatial_orbitals=num_spatial_orbitals,
                                                 num_particles=num_ab_particles,
                                                 qubit_mapper=mapper))

  twolocal_ansatz = TwoLocal(num_qubits=mapped_hamiltonian.num_qubits,
                             rotation_blocks=['rx', 'rz'],
                             entanglement_blocks=['cz'],
                             entanglement="linear",
                             reps=2,
                             initial_state=None)  
  
  return uccsd_ansatz, twolocal_ansatz

In [ ]:
def energy_function(params, ansatz, hamiltonian, estimator, batch, metadata_dict):
  max_circuit = 500
  whole_circuit = []
  for i in range(0, len(ansatz), max_circuit):
    whole_circuit.append(ansatz[i:i + max_circuit])
  jobs = []
  job_ids = []

  with batch:
    for circuit in whole_circuit:
      pub = (circuit, [hamiltonian], [params])
      job = estimator.run([pub])
      jobs.append(job)
      job_id = job.job_id()
      metadata_dict['job_ids'].append(job_id)
      result = job.result()

  energy = result[0].data.evs[0]
  err = result[0].data.stds[0]

  metadata_dict['iters'] += 1
  metadata_dict['params'].append(params)
  metadata_dict['energy'].append((energy, err))

  print(f"Iters. done: {metadata_dict['iters']} [Energy: {energy} +/- {err}]")

  return energy

In [29]:
def run_vqe_twolocal(backend, isa_twolocal, isa_hamiltonian_twolocal):
  num_params_twolocal = isa_twolocal.num_parameters
  x0_twolocal = np.zeros(num_params_twolocal)

  twolocal_dict = {
    "session_id": None,
    "iters": 0,
    "backend": backend.name,
    "maxiter": 200,
    "params": [],
    "energy": [],
    "job_ids": [],
    "job_info": [],
    "start_name": None,
    "end_time": None,
    "completed": False
  }

  batch = Batch(backend=backend, max_time=3600)

  estimator = Estimator(mode=batch)
  estimator.options.default_precision = 1e-1
  estimator.options.resilience_level = 1
  estimator.options.default_shots = 8912

  res = minimize(
    energy_function,
    x0_twolocal,
    args=(
      isa_twolocal,
      isa_hamiltonian_twolocal,
      estimator,
      batch,
      twolocal_dict
    ),
    method="COBYLA",
    tol=1e-1
  )
    
  return twolocal_dict

In [ ]:
def run_vqe_uccsd(backend, isa_uccsd, isa_hamiltonian_uccsd):
  num_params_uccsd = isa_uccsd.num_parameters
  x0_uccsd = np.zeros(num_params_uccsd)

  uccsd_dict = {
    "session_id": None,
    "iters": 0,
    "backend": backend.name,
    "maxiter": 200,
    "params": [],
    "energy": [],
    "job_ids": [],
    "job_info": [],
    "start_time": None,
    "end_time": None,
    "completed": False
  }

  with Batch(backend=backend, max_time=3600) as batch:
    estimator = Estimator(mode=batch)
    estimator.options.default_precision = 1e-1
    estimator.options.resilience_level = 1
    estimator.options.default_shots = 8912

    res = minimize(
      energy_function,
      x0_uccsd,
      args=(
        isa_uccsd,
        isa_hamiltonian_uccsd,
        estimator,
        batch,
        uccsd_dict
      ),
      method="COBYLA",
      tol=1e-1
    )
    
  return uccsd_dict


In [19]:
mapped_hamiltonian, mapper = map_to_qubits(hamiltonian)

uccsd_ansatz, twolocal_ansatz = create_ansatz(mapped_hamiltonian, mapper)

isa_uccsd, isa_hamiltonian_uccsd = transpile_and_apply(uccsd_ansatz,
                                                       mapped_hamiltonian,
                                                       backend,
                                                       optimization_level=3,
                                                       passmanager=pm)

isa_twolocal, isa_hamiltonian_twolocal = transpile_and_apply(twolocal_ansatz,
                                                             mapped_hamiltonian,
                                                             backend,
                                                             optimization_level=3,
                                                             passmanager=pm)

In [ ]:
uccsd_results = run_vqe_uccsd(backend, isa_uccsd, isa_hamiltonian_uccsd)